In [4]:
#Dependencies
import requests
import json
from api_keys import api_key_FBI
import pandas as pd

In [14]:
# the abrebiation of every state.
state_codes = pd.DataFrame({"States":['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT',
                'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
                'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 
                'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 
                'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 
                'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 
                'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']})

data_homicide=state_codes.copy()
data_homicide["Total Homicide"]=""
data_homicide.head()



,States,Total Homicide
0,AL,
1,AK,
2,AZ,
3,AR,
4,CA,


In [24]:
state="AL"
base_url=f"https://api.usa.gov/crime/fbi/cde/shr/state/{state}/victim/sex?from=2021&to=2021&API_KEY={api_key_FBI}"
print(base_url)
response = requests.get(base_url)
response= response.json()

data_homicide.loc[0,"Total Homicide"]=response["data"][0]["Unknown"]+response["data"][0]["Male"]+response["data"][0]["Female"]
data_homicide.head()

https://api.usa.gov/crime/fbi/cde/shr/state/AL/victim/sex?from=2021&to=2021&API_KEY=ePO1sQcPSnSYpfnsdfriGACTxzZScged0XX3tNPm


,States,Total Homicide
0,AL,309
1,AK,
2,AZ,
3,AR,
4,CA,


In [21]:
for index,state in data_homicide.iterrows():
    #with this we get the information of FBI API
    base_url=f"https://api.usa.gov/crime/fbi/cde/shr/state/{state}/victim/sex?from=2021&to=2021&API_KEY={api_key_FBI}"
    #print(base_url)
    response = requests.get(base_url)
    response= response.json()

    data_homicide.loc[index,"Total Homicide"]=response["data"][0]["unknown"]+response["data"][0]["male"]+response["data"][0]["female"]

data_homicide
    


JSONDecodeError: Expecting value: line 1 column 1 (char 0)